In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import constants, units
from astropy.coordinates import Angle
from scipy.optimize import curve_fit
import pandas as pd
import os
from scipy.signal import medfilt
from astropy.wcs import WCS
import scipy.integrate as integrate
from astropy.coordinates import SkyCoord
from gwcs.wcstools import wcs_from_points
import astropy.coordinates as coord

In [5]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/CASBaH/'

In [6]:
plt.style.use('/Users/thepoetoftwilight/Documents/CUBS/Code/science.mplstyle')

Load CASBaH redshifts

In [29]:
df = pd.DataFrame(np.loadtxt(rootdir+'PG1522.txt',dtype='str',delimiter=','))

In [30]:
column_list = ['ID','RA','Dec','z','dz','z_flag','Instrument']
df.columns = column_list

In [34]:
for c in column_list:
    if c != 'Instrument':
        df[c] = pd.to_numeric(df[c])

In [39]:
df

,ID,RA,Dec,z,dz,z_flag,Instrument
0,7503,230.46139,9.39591,0.1806,0.0001,3,SDSS
1,7504,230.71341,9.07406,0.1792,0.0001,3,SDSS
2,7505,230.85305,10.32534,0.4262,0.0002,3,SDSS
3,7506,231.80065,9.54506,0.2225,0.0001,3,SDSS
4,7507,231.25343,10.45284,0.1351,0.0001,3,SDSS
...,...,...,...,...,...,...,...
1052,8555,231.07839,10.04844,0.2179,0.0001,4,SDSS
1053,8556,230.97592,9.52134,0.3763,0.0001,4,SDSS
1054,8557,230.87921,10.02391,0.5451,0.0001,4,SDSS
1055,8558,231.10346,9.97708,0.1202,0.0001,4,LRISb


In [44]:
df_conf = df[((df['z_flag']==4)|(df['z_flag']==3))]

In [45]:
df_conf

,ID,RA,Dec,z,dz,z_flag,Instrument
0,7503,230.46139,9.39591,0.1806,0.0001,3,SDSS
1,7504,230.71341,9.07406,0.1792,0.0001,3,SDSS
2,7505,230.85305,10.32534,0.4262,0.0002,3,SDSS
3,7506,231.80065,9.54506,0.2225,0.0001,3,SDSS
4,7507,231.25343,10.45284,0.1351,0.0001,3,SDSS
...,...,...,...,...,...,...,...
1052,8555,231.07839,10.04844,0.2179,0.0001,4,SDSS
1053,8556,230.97592,9.52134,0.3763,0.0001,4,SDSS
1054,8557,230.87921,10.02391,0.5451,0.0001,4,SDSS
1055,8558,231.10346,9.97708,0.1202,0.0001,4,LRISb


Apply redshift cuts

In [46]:
dv_c = 500/3e5

z_list = [0.67546, 1.04431, 1.09457, 1.16591, 1.22541, 1.25937, 1.27767]

z_min_list = []
z_max_list = []

for z in z_list:
    
    z_min_list.append(z-(1+z)*dv_c)
    z_max_list.append(z+(1+z)*dv_c)

In [48]:
np.round(z_min_list,5)

array([0.67267, 1.0409 , 1.09108, 1.1623 , 1.2217 , 1.2556 , 1.27387])

In [49]:
np.round(z_max_list,5)

array([0.67825, 1.04772, 1.09806, 1.16952, 1.22912, 1.26314, 1.28147])

In [62]:
df_conf[df_conf['z']>0.6]

,ID,RA,Dec,z,dz,z_flag,Instrument
26,7529,230.65691,9.88108,0.6118,0.0003,3,SDSS
58,7561,231.38632,9.74586,0.6294,0.0002,3,SDSS
159,7662,230.88427,9.57746,0.6055,0.0002,3,SDSS
203,7706,230.95688,9.94632,1.2114,0.0002,3,SDSS
214,7717,231.20121,9.91829,0.6041,0.0002,3,SDSS
226,7729,231.36164,10.23552,0.6049,0.0002,3,SDSS
310,7813,231.17555,9.72156,0.6040,0.0002,3,SDSS
333,7836,231.27865,9.69681,0.6559,0.0001,3,SDSS
336,7839,231.18906,9.89420,0.6535,0.0002,3,SDSS
348,7851,231.10557,10.25697,0.7872,0.0002,3,SDSS


In [53]:
df_conf[(df_conf['z']>0.67267)&(df_conf['z']<0.67825)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [54]:
df_conf[(df_conf['z']>1.0409)&(df_conf['z']<1.04772)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [55]:
df_conf[(df_conf['z']>1.09108)&(df_conf['z']<1.09806)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [57]:
df_conf[(df_conf['z']>1.1623)&(df_conf['z']<1.16952)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [58]:
df_conf[(df_conf['z']>1.2217)&(df_conf['z']<1.22912)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [59]:
df_conf[(df_conf['z']>1.2556)&(df_conf['z']<1.26314)]

,ID,RA,Dec,z,dz,z_flag,Instrument


In [60]:
df_conf[(df_conf['z']>1.27387)&(df_conf['z']<1.28147)]

,ID,RA,Dec,z,dz,z_flag,Instrument
